# Group 11

## Iben Mai Huse, Karlis Buiko, Markus Sibbesen, Ida Kathrine Jensen

This notebook contains the code for data exploration, and will be the hand-in for the First Year Project 4

Created 04.05.2021

Last modified, modified by:

04.05.2021, Ida

### Imports

### Path to files

## Task 1: Preprocessing

#### We have selected *hate speech detection* as the binary classifier and the *sentiment analysis* as the multiclass classifier.

*Using regular expressions, implement a tokeniser to split the input texts into meaningful tokens. Your tokeniser should be a script that takes as input the data as distributed in the dataset and outputs the tokenised text, one output line per input line, with spaces between tokens.*

*• Use the training set of one of the tasks you’ve chosen to work on. Set aside a part of the training set for evaluating the tokeniser so you don’t have to touch the validation data for that.*

*• Start by taking a subset of the trainingdata, look through it and discuss in your group what a good tokenisation of this subset should look like. Then design your initial Python implementation to match this ideal tokenisation, and run it over the portion of the training set that you didn’t hold out for tokeniser evaluation. Keep an eye on infrequently occurring tokens in the output that look like tokenisation errors.*

*• Once you’re done, compare your tokeniser’s output with the baseline tokenisation
you get from the social media tokeniser in the NLTK library (nltk.tokenize.TweetTokenizer), using the data you’ve set aside for this purpose.*

*
• Consider using the difflib package in Python, or the diff utility in the Unix shell, to compare the output of the two tokenisers efficiently.*

## Task 2: Characterising Your Data

*Characterise the training sets of the two tasks you’ve chosen in terms of elementary corpus statistics: <br>
• Corpus size, vocabulary size,type/token ratio.<br>
• What are the most frequent tokens?<br>
• What types of tokens occur only once, or 2 or 3 times?<br>
• Are there any noticeable differences between your two datasets?<br>
• Are the corpus statistics consistent with Zipf’slaw?(no formal test needed,but a plot would be helpful)*<br>

## Task 3: Manual Annotation and Inter-Annotator Agreement

*Choose one of your two datasets. For this subtask, the emoji prediction dataset doesn’t make sense, so if that is one of your choices, pick the other one for this part of the assignment.*

*In the README file in the TweetEval corpus repository, there are links to research papers from the SemEval workshop, describing how each of the datasets was created and annot- ated. Locate the one that belongs to the dataset you’ve picked for this subtask and find the passages that describe in detail how the labels for the dataset were created. Read these passages carefully.*

*Next, select a random sample of 100 tweets from the training set. Working independently from each other and without consulting the labels published in the TweetEval corpus, each member of your group should now go manually through this sample and label them ac- cording to the same scheme.*

*Report on the inter-annotator agreement, including the agreement with the published la- bels, and discuss what phenomena in the data caused the biggest problems for inter-an- notator agreement.*

## Task 4: Automatic Prediction

*Finally, use scikit-learn to train a classifier for the automatic prediction of the labels in the two datasets you have chosen. During the lessons, we have not had time to discuss ma- chine learning techniques and classification methods in detail, so in this exercise you will be using library implementations as “black box” methods.* <br>

*Run all classification experiments on both of the tasks you’ve chosen (one binary and one multi-class task). Evaluate your different classifiers on the validation set and report relev- ant evaluation metrics (accuracy, precision/recall/F-score).* <br>

*As a baseline, start with the sklearn.linear_model.SGDClassifier in a logistic regres- sion configuration (loss=’log’) using bag of words features. Then run at least additional experiments trying to improve your initial scores by any means you can think of. Try out at least 4 different methods. Usually you will need to run several experiments for each meth- ods to test different parameter values. Here are some settings in scikit-learn that you could experiment with:*

*• Additional preprocessing options: n-gram features, lowercasing, stop word lists (see options to sklearn.feature_extraction.text.CountVectorizer)*. <br>

*• Count transformations (sklearn.feature_extraction.text.TfIdfTransformer).* <br>

*• The classification loss(lossparametertoSGDClassifier).* <br>

*• The regularisation strength (alpha parameter to SGDClassifier – try varying it in exponentially spaced steps).* <br>

*• Different classifiers (e.g.,sklearn.ensemble.Random Forest Classifier or sklearn.naive_bayes.MultinomialNB.* <br>

*• Anything else discussed during the lessons,or implemented in scikit-learn.*

*(For the systems that achieves the highest accuracy on the validation set, run the evaluation on the test set and report your results. We will share an anonymous overview of the test set scores of all groups after the reports are handed in.)*

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5235cf77-d347-437f-a336-60ef81766ce7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>